In [8]:
#load libraries 
import pandas as pd
import altair as alt
import numpy as np

stadium = pd.read_csv("clean_stadium.csv")
fanbase = pd.read_csv("clean_fanbase.csv")
merch = pd.read_csv("clean_merch.csv")

In [24]:
age_category_revenue = merged_spending.groupby(
    ['Final_Age_Group', 'Item_Category'], 
    as_index=False
)['Unit_Price'].sum()

age_category_revenue.columns = ['Age_Group', 'Item_Category', 'Revenue']

heatmap_pivot = age_category_revenue.pivot(
    index='Age_Group',
    columns='Item_Category',
    values='Revenue'
).fillna(0)

# sort age groups
age_order = ['<18', '18-25', '26-40', '41-60', '60+']
heatmap_pivot = heatmap_pivot.reindex(age_order)

# transform into long form
heatmap_long = heatmap_pivot.reset_index().melt(
    id_vars='Age_Group',
    var_name='Item_Category',
    value_name='Revenue'
)

# viz 
base = alt.Chart(heatmap_long).encode(
    x=alt.X('Age_Group:N', 
            title='Age Group',
            sort=age_order,
            axis=alt.Axis(labelAngle=0, labelFontSize=12)),
    y=alt.Y('Item_Category:N', 
            title='Item Category',
            axis=alt.Axis(labelFontSize=11))
)

heatmap = base.mark_rect().encode(
    color=alt.Color('Revenue:Q',
                    scale=alt.Scale(scheme='oranges'),
                    title='Revenue ($)',
                    legend=alt.Legend(format='$,.0s')),
    tooltip=[
        alt.Tooltip('Age_Group:N', title='Age Group'),
        alt.Tooltip('Item_Category:N', title='Category'),
        alt.Tooltip('Revenue:Q', title='Revenue', format='$,.0f')
    ]
)

text = base.mark_text(
    baseline='middle',
    fontSize=11).encode(
    text=alt.Text('Revenue:Q', format='$,.0f'),
    color=alt.condition(
        'datum.Revenue > ' + str(heatmap_long['Revenue'].max() * 0.5),
        alt.value('white'),
        alt.value('black')))

# add layers
chart = (heatmap + text).properties(
    width=700,
    height=350,
    title='Age Group × Item Category Revenue Heatmap').configure_axis(
    labelFontSize=12,
    titleFontSize=13).configure_title(
    fontSize=16,
    fontWeight='bold')

chart

alt.LayerChart(...)